In [6]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

1.10.0


In [7]:
CSV_COLUMNS = ['radius','height','volume']
LABEL_COLUMN = 'volume'
DEFAULTS = [[0.0], [0.0], [0.0]]

def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label

    # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
    filenames_dataset = tf.data.Dataset.list_files(filename)
    # Read lines from text files
    textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
    # Parse text lines as comma-separated values (CSV)
    dataset = textlines_dataset.map(decode_csv)
    
    # Note:
    # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
    # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # loop indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn
    

def get_train():
  return read_dataset('./volume-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid():
  return read_dataset('./volume-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
  return read_dataset('./volume-test.csv', mode = tf.estimator.ModeKeys.EVAL)

In [8]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('radius'),
    tf.feature_column.numeric_column('height'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

In [11]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'volume_trained3'
#TensorBoard().start(OUTDIR)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns = feature_cols, model_dir = OUTDIR)
model.train(input_fn = get_train(), max_steps = 3000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_session_config': None, '_evaluation_master': '', '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000012539EF0>, '_task_id': 0, '_global_id_in_cluster': 0, '_keep_checkpoint_max': 5, '_device_fn': None, '_is_chief': True, '_master': '', '_save_checkpoints_secs': 600, '_task_type': 'worker', '_model_dir': 'volume_trained3'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into volume_trained3\model.ckpt.
INFO:t

In [12]:
def print_rmse(model, name, input_fn):
  metrics = model.evaluate(input_fn = input_fn, steps = None)
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'validation', get_valid())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-23-06:30:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from volume_trained3\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-23-06:30:23
INFO:tensorflow:Saving dict for global step 3000: average_loss = 7.627667e+17, global_step = 3000, label/mean = 583032100.0, loss = 3.8138332e+20, prediction/mean = 43468.848
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: volume_trained3\model.ckpt-3000
RMSE on validation dataset = 873365120.0


In [13]:
print_rmse(model, 'validation', get_test())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-23-06:30:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from volume_trained3\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-23-06:30:26
INFO:tensorflow:Saving dict for global step 3000: average_loss = 1.7619267e+21, global_step = 3000, label/mean = 30846015000.0, loss = 3.224326e+23, prediction/mean = 47899.168
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: volume_trained3\model.ckpt-3000
RMSE on validation dataset = 41975308288.0
